In [37]:
import pandas as pd
import numpy as np

In [58]:
df

,A,B,group,number,link,bus,dir
0,8133,8127,SL1,1,洲美快速道路,0,出市中心
1,8127,8131,SL1,1,洲美快速道路,0,進市中心
2,6666,8115,SL1,2,百齡橋,0,出市中心
3,6637,6666,SL1,2,百齡橋,0,進市中心
4,8292,10246,SL1,3,承德橋,0,出市中心
...,...,...,...,...,...,...,...
263,9603,9608,SL13,3,臺64平面段,0,往東
264,13788,9533,SL13,4,縣106_中山路三段,0,往西
265,9533,13788,SL13,4,縣106_中山路三段,0,往東
266,9694,9610,SL13,5,連城路,0,往西


In [66]:
#read TRTS4S_屏柵AB點.xlsx sheet 工作表2
# columns G, H as group, number, column A, B as A,B; row 2-269
df = pd.read_excel('TRTS4S_屏柵AB點.xlsx', sheet_name='工作表2', usecols='G,H,A,B,I,J,L', skiprows=0)
df.columns = ['A', 'B','group', 'number', 'link','bus','dir']

#read TRTS4S_屏柵線.xlsx sheet 節點編號
# columns A, I as group, number, column (D,E) and (G,H) as A,B; row 2-81
df2 = pd.read_excel('TRTS-4S屏柵線.xlsx', sheet_name='節點編號', usecols='A,I,D,E,G,H', skiprows=0)
df2.columns = ['group', 'A1', 'B1', 'A2', 'B2', 'number']
#type int
df2['A1'] = df2['A1'].astype(int)
df2['B1'] = df2['B1'].astype(int)
df2['A2'] = df2['A2'].astype(int)
df2['B2'] = df2['B2'].astype(int)
df2['number'] = df2['number'].astype(int)

#make two dictionaries to compare
dict1={}
for _, row in df.iterrows():
    if (row['group'], row['number']) not in dict1:
        # Avoid overwriting if the key already exists
        dict1[(row['group'], row['number'])] = [(row['A'], row['B'])]
    else:
        dict1[(row['group'], row['number'])].append((row['A'], row['B']))
dict1n = {}
for _, row in df.iterrows():
    if (row['group'], row['number'], row['link'], row['bus'], row['dir']) not in dict1n:
        # Avoid overwriting if the key already exists
        dict1n[(row['group'], row['number'], row['link'], row['bus'], row['dir'])] = [(row['A'], row['B'])]
    else:
        dict1n[(row['group'], row['number'], row['link'], row['bus'], row['dir'])].append((row['A'], row['B']))
dict1e = {}
for _, row in df.iterrows():
    if row['bus'] == 0: # Only consider bus 0 for dict1e
        if (row['group'], row['link']) not in dict1e:
            # Avoid overwriting if the key already exists
            dict1e[(row['group'], row['link'])] = [row['dir'], row['A'], row['B']]
        else:
            dict1e[(row['group'], row['link'])]+= [row['dir'], row['A'], row['B']]
dict2 = {}
for _, row in df2.iterrows():
    if (row['group'], row['number']) not in dict2:
        # Avoid overwriting if the key already exists
        dict2[(row['group'], row['number'])] = [(row['A1'], row['B1']), (row['A2'], row['B2'])]
    else:
        dict2[(row['group'], row['number'])].append((row['A1'], row['B1']))

        dict2[(row['group'], row['number'])].append((row['A2'], row['B2']))

In [67]:
for key, value in dict1e.items():
    if len(value) != 6:
        print(f"Key {key} has {len(value)} values: {value}")

Key ('SL2', '環河北路平面(往北)') has 3 values: ['往北', 6442, 6550]
Key ('SL4', '仁愛路(快車道)') has 3 values: ['往西', 6342, 8478]
Key ('SL4', '信義路(快車道)') has 3 values: ['往東', 8500, 5259]
Key ('SL4', '信義路(慢車道)') has 9 values: ['往東', 10202, 7808, '往東', 10205, 7805, '往東', 10204, 7806]


In [77]:
#output dict1e as excel
list1e = []
for key, value in dict1e.items():
    l=[key[0], key[1]]
    l+=list(value)
    list1e.append(l)
df1e = pd.DataFrame(list1e)
df1e.columns = ['group', 'link', 'dir', 'A1', 'B1','dir', 'A2', 'B2', 'dir', 'A3', 'B3']
df1e.to_excel('dict1e.xlsx', index=False)

In [ ]:
# for all link in dict1n, make a excel 
# group	link    dir A1	B1  dir	A2	B2	number


In [ ]:
#compare the two dictionaries
count = 0
for key in dict1:
    if key in dict2:
        if set(dict1[key]) != set(dict2[key]):
            print(f"Mismatch for {key}: {dict1[key]} != {dict2[key]}")
            count += 1
    else:
        # print(f"Key {key} not found in second dictionary")
        pass
print(f"Total mismatches: {count}")

Mismatch for ('SL1', 3): [(8292, 10246), (10248, 8292), (10247, 8292), (8292, 10245)] != [(10246, 6631), (6631, 10247), (10245, 6631), (6631, 10248)]
Mismatch for ('SL1', 4): [(6676, 6675), (6675, 6676)] != [(6680, 6676), (27273, 6680)]
Mismatch for ('SL1', 7): [(10219, 6294), (10220, 6294), (6294, 10219), (6294, 10218)] != [(10219, 6798), (6798, 10219), (10218, 6798), (6798, 10220)]
Mismatch for ('SL2', 2): [(6442, 6550)] != [(6508, 6513), (6513, 6508)]
Mismatch for ('SL2', 3): [(14086, 6439), (6439, 14086)] != [(6412, 6509), (6509, 6412)]
Mismatch for ('SL2', 4): [(6438, 6437), (6437, 6438)] != [(6438, 6418), (6418, 6438)]
Mismatch for ('SL2', 5): [(6536, 9103), (9103, 6536)] != [(7956, 7957), (7957, 7956)]
Mismatch for ('SL2', 6): [(8372, 8369), (8373, 8370), (9105, 6448), (9133, 8371)] != [(6448, 6420), (8375, 8372), (8371, 8376), (8374, 8373)]
Mismatch for ('SL2', 7): [(9169, 6575), (6575, 9169)] != [(9169, 9031), (9031, 9169)]
Mismatch for ('SL2', 8): [(6570, 6576), (6576, 6570)]

In [30]:
#read network.csv
network_df = pd.read_csv('network.csv', usecols=['A', 'B'])
AB_pairs = set(zip(network_df['A'], network_df['B']))

not_found_pairs = []
#for all A,B in dict1, check if they are in network
for key, value in dict1.items():
    for a, b in value:
        if (a, b) not in AB_pairs:
            print(f"Pair ({a}, {b}) from {key} not found in network.csv")
            not_found_pairs.append((key, a, b))
if not_found_pairs:
    print(f'{len(not_found_pairs)} pairs not found in network.csv:')
    # for key, a, b in not_found_pairs:
        # print(f"Key: {key}, A: {a}, B: {b}")

Pair (8127, 8131) from ('SL1', 1) not found in network.csv
Pair (6676, 6675) from ('SL1', 4) not found in network.csv
Pair (6675, 6676) from ('SL1', 4) not found in network.csv
Pair (9182, 9181) from ('SL2', 14) not found in network.csv
Pair (8662, 8661) from ('SL2', 22) not found in network.csv
Pair (8661, 8662) from ('SL2', 22) not found in network.csv
Pair (6932, 6977) from ('SL2', 23) not found in network.csv
Pair (6977, 6932) from ('SL2', 23) not found in network.csv
Pair (8500, 5259) from ('SL4', 15) not found in network.csv
Pair (5047, 7684) from ('SL5', 4) not found in network.csv
Pair (7684, 5047) from ('SL5', 4) not found in network.csv
Pair (5050, 5051) from ('SL5', 6) not found in network.csv
Pair (5044, 5045) from ('SL5', 6) not found in network.csv
Pair (6514, 5798) from ('SL7', 5) not found in network.csv
Pair (5799, 5303) from ('SL7', 5) not found in network.csv
Pair (9796, 6104) from ('SL8', 8) not found in network.csv
Pair (6104, 9796) from ('SL8', 8) not found in net